In [21]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import re
import requests as req


client = MongoClient('mongodb://localhost:27017/')
db = client.companies


#### First of all, let's query the "young" (valid) companies from our new database:

In [23]:
young=db.offices_ok.find({"is_young":1},{"_id":0,"index":0,'is_young':0,'is_design':0,'is_similar':0,'is_succesful':0})
mydf=pd.DataFrame(young)

In [27]:
mydf=mydf[['name','category_code','description','tag_list','founded_year','number_of_employees','raised_money_dollars','lat','lng','geo_point']]
mydf.shape

(3504, 10)

#### Now we want to make a geo query for each of our desired companies. We would like to know for each company:
#### 1. How many nearby design, similar and/or succesufl companies there are.
#### 2. How many nearby "old" companies there are.

In [30]:
def find_old(df1, radio_max_meters=2000):
    lista=[]
    for i in range(len(df1)): 
        old=db.offices_ok.find({'$and':[{
                        "geo_point": {
                            "$near": {
                                "$geometry": df1[i],
                                "$maxDistance": radio_max_meters,
                            }
                        }
                    },{
                        'is_young':0
                    }]})
        tempdf=pd.DataFrame(old)
        lista.append(tempdf.shape[0])
    return lista
       


In [31]:
def find_young(df1, radio_max_meters=2000):
    lista=[]
    for i in range(len(df1)): 
        old=db.offices_ok.find({'$and':[{
                        "geo_point": {
                            "$near": {
                                "$geometry": df1[i],
                                "$maxDistance": radio_max_meters,
                            }
                        }
                    },{
                        'is_young':1
                    }]})
        tempdf=pd.DataFrame(old)
        lista.append(tempdf.shape[0])
    return lista

In [32]:
def find_design(df1, radio_max_meters=2000):
    lista=[]
    for i in range(len(df1)): 
        old=db.offices_ok.find({'$and':[{
                        "geo_point": {
                            "$near": {
                                "$geometry": df1[i],
                                "$maxDistance": radio_max_meters,
                            }
                        }
                    },{
                        'is_design':1
                    }]})
        tempdf=pd.DataFrame(old)
        lista.append(tempdf.shape[0])
    return lista

In [33]:
def find_similar(df1, radio_max_meters=2000):
    lista=[]
    for i in range(len(df1)): 
        old=db.offices_ok.find({'$and':[{
                        "geo_point": {
                            "$near": {
                                "$geometry": df1[i],
                                "$maxDistance": radio_max_meters,
                            }
                        }
                    },{
                        'is_similar':1
                    }]})
        tempdf=pd.DataFrame(old)
        lista.append(tempdf.shape[0])
    return lista

In [34]:
def find_succesful(df1, radio_max_meters=2000):
    lista=[]
    for i in range(len(df1)): 
        old=db.offices_ok.find({'$and':[{
                        "geo_point": {
                            "$near": {
                                "$geometry": df1[i],
                                "$maxDistance": radio_max_meters,
                            }
                        }
                    },{
                        'is_succesful':1
                    }]})
        tempdf=pd.DataFrame(old)
        lista.append(tempdf.shape[0])
    return lista

In [36]:
mydf["old_near"]=find_old(mydf["geo_point"])
mydf["young_near"]=find_young(mydf["geo_point"])
mydf["design_near"]=find_design(mydf["geo_point"])
mydf["similar_near"]=find_similar(mydf["geo_point"])
mydf["succesful_near"]=find_succesful(mydf["geo_point"])

In [38]:
mydf.head(3)

,name,category_code,description,tag_list,founded_year,number_of_employees,raised_money_dollars,lat,lng,geo_point,old_near,young_near,design_near,similar_near,succesful_near
0,Geni,web,Geneology social network site,"geni, geneology, social, family, genealogy",2006,18,16500000,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,...",0,8,1,8,2
1,MeetMoi,social,Mobile Dating,"mobile, dating, location, realtime, phone",2007,15,5580000,40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ...",33,104,3,81,37
2,Twitter,social,Real time communication platform,"text, messaging, social, community, twitter, t...",2006,1300,1160000000,37.776805,-122.416924,"{'type': 'Point', 'coordinates': [-122.4169244...",19,109,3,83,53


#### Now let's keep only the companies that have no "old" companies around them and create a "score" field for them. This score will be based in normalizing the *design_near, similar_near and succesful_near* columns based on their maximum values and then summing their values for each record. Each variable will have the same weight and the maximum score will be 3:

In [231]:
young=mydf[mydf["old_near"]==0]

In [233]:
young = young.drop(["description","tag_list","geo_point","old_near","young_near"], axis=1)

In [234]:
young["Score1"]=(young["design_near"]/young["design_near"].max())+(young["similar_near"]/young["similar_near"].max())+(young["succesful_near"]/young["succesful_near"].max())

#### Now let's rank the companies by their score and keep the top 100:

In [235]:
young=young.sort_values(by='Score1', ascending=False)
young=young.reset_index()


In [236]:
top100=young.copy()[:100]
top100.head(3)

,index,name,category_code,founded_year,number_of_employees,raised_money_dollars,lat,lng,design_near,similar_near,succesful_near,Score1
0,844,SocialVibe,advertising,2007,80,43900000,34.081524,-118.382674,2,14,2,2.183333
1,2192,Silver Tail Systems,analytics,2008,90,22100000,37.428088,-122.143368,0,15,8,2.000000
2,129,SayNow,mobile,2005,20,7500000,37.427460,-122.143915,0,15,8,2.000000


#### Now we want to know how these locations match the preferences of our employees. We will use the google maps API to figure it out:
#### 1. The number of schools in a radius of 1000m.
#### 2. Number of starbucks in a radius of 200 m.
#### 3. Number of airports in a radius of 10000 m.
#### 4. Number of bars in a radius of 500m.
#### 5. Number of vegan restaurant in a radius of 200m.


In [237]:
import os
from dotenv import load_dotenv
load_dotenv()

if not "PLACES_KEY" in os.environ:
    raise ValueError("You should pass a PLACES_KEY, see: https://developers.google.com/places/web-service/get-api-key")

PLACES_KEY = os.environ["PLACES_KEY"]

def get_near(df,tipo,keyword,radius):
    lst=[]
    for i in range(len(df)):
        lat=df["lat"][i]
        lng=df["lng"][i]
        res = req.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&type={}&keyword={}&key={}".format(lat,lng,radius,tipo,keyword,PLACES_KEY))
        content=res.json()
        if content['status']=='OK': lst.append(len(content["results"]))
        else: lst.append(0)
    return lst


In [238]:
top100["schools"]=get_near(top100,'school','school',1000)

In [239]:
top100["starbucks"]=get_near(top100,'cafe','starbucks',200)

In [240]:
top100["bars"]=get_near(top100,'bar','bar',500)

In [241]:
top100["vegan_rests"]=get_near(top100,'restaurant','vegan',200)

In [242]:
top100["airports"]=get_near(top100,'airport','international%20airport',10000)

In [205]:
top100 = top100.drop(["index","description","tag_list","geo_point","old_near","young_near"], axis=1)

#### Now we will create another score column that will score each company based on "likes" of the employees. It will be calculated the same way as score 1 but using the info requested from the API and assigning different weights to each variable (*schools 0.6, airports 0.5, bars 0.4, starbucks 0.25 and vegan restaurants 0.25*), the maximum value of this score will be 2:

In [247]:
top100["Score2"]=(0.6*top100["schools"]/top100["schools"].max())+(0.25*top100["starbucks"]/top100["starbucks"].max())+(0.4*top100["bars"]/top100["bars"].max())+(0.25*top100["vegan_rests"]/top100["vegan_rests"].max())+(0.5*top100["airports"]/top100["airports"].max())

#### Also, let's create a total score that will be the sum of both scores and sort the list by total score:

In [249]:
top100["Total_score"]=top100["Score1"]+top100["Score2"]

In [298]:
top100=top100.sort_values(by='Total_score', ascending=False)
top100=top100.reset_index()

,level_0,index,name,category_code,founded_year,number_of_employees,raised_money_dollars,lat,lng,design_near,similar_near,succesful_near,Score1,schools,starbucks,bars,vegan_rests,airports,Score2,Total_score
0,0,844,SocialVibe,advertising,2007,80,43900000,34.081524,-118.382674,2,14,2,2.183333,10,1,20,2,3,0.922078,3.105411
1,3,107,Doostang,web,2005,10,5750000,37.427235,-122.145783,0,15,8,2.000000,13,2,8,9,2,0.901623,2.901623
2,2,129,SayNow,mobile,2005,20,7500000,37.427460,-122.143915,0,15,8,2.000000,15,1,8,8,2,0.893766,2.893766
3,1,2192,Silver Tail Systems,analytics,2008,90,22100000,37.428088,-122.143368,0,15,8,2.000000,15,1,7,8,2,0.873766,2.873766
4,6,1506,plaYce,games_video,2008,7,0,37.802040,-122.438231,1,11,3,1.608333,20,3,20,2,0,1.185714,2.794048


In [299]:
top100 = top100.drop(["index","level_0"], axis=1)
top100.head()

,name,category_code,founded_year,number_of_employees,raised_money_dollars,lat,lng,design_near,similar_near,succesful_near,Score1,schools,starbucks,bars,vegan_rests,airports,Score2,Total_score
0,SocialVibe,advertising,2007,80,43900000,34.081524,-118.382674,2,14,2,2.183333,10,1,20,2,3,0.922078,3.105411
1,Doostang,web,2005,10,5750000,37.427235,-122.145783,0,15,8,2.000000,13,2,8,9,2,0.901623,2.901623
2,SayNow,mobile,2005,20,7500000,37.427460,-122.143915,0,15,8,2.000000,15,1,8,8,2,0.893766,2.893766
3,Silver Tail Systems,analytics,2008,90,22100000,37.428088,-122.143368,0,15,8,2.000000,15,1,7,8,2,0.873766,2.873766
4,plaYce,games_video,2008,7,0,37.802040,-122.438231,1,11,3,1.608333,20,3,20,2,0,1.185714,2.794048


####  Here's a heat map of our top 100 companies:

In [300]:
import folium
from folium import plugins 

m = folium.Map(zoom_start=5)
stationArr = top100[['lat', 'lng']][0:100].values
m.add_child(plugins.HeatMap(stationArr, radius=15))
m

#### And here we can see how well each company did based on their score represented by the size of their radius:

In [301]:
m = folium.Map(zoom_start=5)

for i in range(len(top100["lat"])):
    tooltip = top100["name"][i]
    loc=[top100["lat"][i], top100["lng"][i]]
    score=int(top100["Total_score"][i])
    folium.Marker(loc, popup=tooltip, tooltip=tooltip).add_to(m)
    folium.Circle(location=loc, radius=(score)*100000, fill=True, fill_color='blue').add_to(m)


In [302]:
m

#### Now let's have a look at our top 10, we will choose our offices location based on these:

In [303]:
top10=top100.copy().head(10)
top10

,name,category_code,founded_year,number_of_employees,raised_money_dollars,lat,lng,design_near,similar_near,succesful_near,Score1,schools,starbucks,bars,vegan_rests,airports,Score2,Total_score
0,SocialVibe,advertising,2007,80,43900000,34.081524,-118.382674,2,14,2,2.183333,10,1,20,2,3,0.922078,3.105411
1,Doostang,web,2005,10,5750000,37.427235,-122.145783,0,15,8,2.000000,13,2,8,9,2,0.901623,2.901623
2,SayNow,mobile,2005,20,7500000,37.427460,-122.143915,0,15,8,2.000000,15,1,8,8,2,0.893766,2.893766
3,Silver Tail Systems,analytics,2008,90,22100000,37.428088,-122.143368,0,15,8,2.000000,15,1,7,8,2,0.873766,2.873766
4,plaYce,games_video,2008,7,0,37.802040,-122.438231,1,11,3,1.608333,20,3,20,2,0,1.185714,2.794048
5,Boxbe,web,2005,3,1500000,37.800209,-122.442592,1,12,3,1.675000,20,1,20,1,0,1.067857,2.742857
6,PerfectBusiness,web,2008,10,0,33.995974,-118.456709,2,7,2,1.716667,11,1,2,1,11,0.937857,2.654524
7,IGOpeople,network_hosting,2008,4,0,53.336139,-6.249789,1,9,2,1.350000,20,3,19,1,3,1.284221,2.634221
8,ftopia,software,2009,7,0,53.332814,-6.249378,1,9,2,1.350000,20,3,14,3,1,1.129026,2.479026
9,Making Sense,software,2006,100,0,-34.589281,-58.432970,2,3,0,1.200000,20,1,20,3,2,1.194481,2.394481


In [306]:
m = folium.Map(zoom_start=5)

for i in range(len(top10["lat"])):
    tooltip = top10["name"][i]
    loc=[top10["lat"][i], top10["lng"][i]]
    score=int(top10["Total_score"][i])
    folium.Marker(loc, popup=tooltip, tooltip=tooltip).add_to(m)

In [307]:
m

#### Based on the score